In [34]:
import praw

# Reddit API credentials
reddit = praw.Reddit(client_id="2KDziNafg0HghUP0CMzCFQ",         # your client id
                               client_secret="-LWE8i0kVizEhsKAj-FbANFb8diiow",      # your client secret
                               user_agent="zelda")        # your user agent
submission = reddit.submission("3g1jfi")
submission.comments.replace_more(limit=None)
comment_queue = submission.comments[:]  # Seed with top-level
while comment_queue:
    print("**********************************")
    comment = comment_queue.pop(0)
    print(comment.body)
    print(comment.report_reasons)
    comment_queue.extend(comment.replies)
    print("**********************************")

Fetching: GET https://oauth.reddit.com/comments/3g1jfi/ at 1730103514.713095
Data: None
Params: {'limit': 2048, 'raw_json': 1, 'sort': 'confidence'}
Response: 200 (75238 bytes) (rst-83:rem-989.0:used-11 ratelimit) at 1730103517.77576
Fetching: POST https://oauth.reddit.com/api/morechildren/ at 1730103517.8099284
Data: [('api_type', 'json'),
 ('children',
  'cyikrzr,ctu3jqd,ctu1j06,ctudfza,ctu8dct,ctud1hd,ctuc2pt,ctu5sjx,ctu0791,ctud8dw,ctu0lq1,ctu85uv,ctu38pi,ctubqz6,ctu11wy,ctuciai,ctu4s23,ctu8qmv,ctuc5sn,ctu44um,ctuba3k,ctu0mk8,ctu899f'),
 ('link_id', 't3_3g1jfi'),
 ('sort', 'confidence')]
Params: {'raw_json': 1}
Response: 200 (5187 bytes) (rst-81:rem-988.0:used-12 ratelimit) at 1730103518.2633317
Fetching: POST https://oauth.reddit.com/api/morechildren/ at 1730103518.2687838
Data: [('api_type', 'json'),
 ('children',
  'ctuc6vw,ctuceoh,cvcrc69,ctucpmm,ctuf0yj,ctucymm,ctul9rx,ctuyak2,ctudgos,ctums31,cwu0a4e,ctue7tu,ctur0ss,cu0ia8m,ctuecwj,ctuf4hv,ctutgbt,ctzz90y,ctui5nm,ctukmvi,culkv

KeyboardInterrupt: 

## Posts_data as a Graph

In [8]:
import networkx as nx

def create_graph_from_posts_data(posts_data):
    """
    Creates a NetworkX graph from the posts_data structure.

    Args:
        posts_data: A list of dictionaries, each representing a post with its comments.

    Returns:
        A NetworkX DiGraph representing the posts and comment relationships.
    """

    graph = nx.DiGraph()

    for post_data in posts_data:
        post_id = post_data['post_id']
        # Add post node
        graph.add_node(post_id, 
                       type='post', 
                       subreddit=post_data['subreddit'], 
                       title=post_data['post_title'], 
                       author=post_data['post_author'], 
                       text=post_data['post_text'],
                    #    score=post_data['post_upvote_score'],
                    #    created_utc=post_data['post_created_utc']
                       )

        comment_graph = post_data['comments_graph']
        for comment_id, comment_data in comment_graph.items():
            # Add comment node
            try:
                graph.add_node(comment_id, 
                            type='comment', 
                            author=comment_data['author'], 
                            body=comment_data['body'],
                            # Add other comment attributes as needed
                            )
                # Add edge from post to comment
                graph.add_edge(post_id, comment_id)

                # Add edges between comments based on parent_id
                if comment_data['parent_id']:
                    graph.add_edge(comment_data['parent_id'], comment_id)
            except:
                pass
    return graph



from opinion_leaders_reddit.analysing import OpinionLeaderDataPreparator as oldp

# load the data
path = "data/raw_data/20241107_130229_posts_data_test__worldnews"
oldp.load_posts_data(filename=path)



# Create NetworkX graph
graph = create_graph_from_posts_data(oldp.posts_data)

In [11]:
import networkx as nx
import plotly.graph_objects as go



# Visualization with Plotly
pos = nx.spring_layout(graph)  # Layout algorithm

edge_x = []
edge_y = []
for edge in graph.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
node_text = []
node_color = []
for node in graph.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    if graph.nodes[node]['type'] == 'post':
        # node_text.append(f"Post: {graph.nodes[node]['title']}")
        node_color.append('blue')  # Color for post nodes
    else:
        # node_text.append(f"Comment:" # {graph.nodes[node]['body'][:30]}..."
        #                  )
        node_color.append('red')  # Color for comment nodes

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers+text',
    hoverinfo='text',
    text=node_text,
    marker=dict(
        showscale=False,
        colorscale='YlGnBu',
        color=node_color,
        size=10,
        line_width=2))

fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='Reddit Post and Comment Network',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

#### Nested Comments and their level


create a class that taken a numer of terms in different languages, return a graph structure that contains post and comments for each post that contains at least on of this terms (this means at comment or post level)

Class():
    Init
        - reddit configuration 
        - search terms (linguage: list) to run on all Reddit 

    SearchPostsAndComments():
        - for post in posts:
          - extract 
            -     'subreddit': submission.subreddit.display_name,
                'post_id': submission.id,
                'post_title': submission.title.encode('utf-8').decode('utf-8'),
                'post_author': submission.author.name if submission.author else None,  # Handle deleted authors
                'post_text': submission.selftext.encode('utf-8').decode('utf-8'),
                'comments_bodies': [],
                'comments_lenght': [],
                'comments_author': [],
                'comments_upvote_score': [],
                'comments_created_utc': [],
                'comments_num': 0,
                'post_created_utc': datetime.utcfromtimestamp(submission.created_utc).isoformat(),
                'post_upvote_score': submission.score,
                'post_upvote_ratio': submission.upvote_ratio,
                'post_url': submission.url, 
                # 'post_language': language, 
            - run nested search on comments

In [27]:
comment.report_reasons

In [10]:
from datetime import datetime
submission = reddit.submission("3g1jfi")

post_data = {
    'subreddit': submission.subreddit.display_name,
    'post_id': submission.id,
    'post_title': submission.title.encode('utf-8').decode('utf-8'),
    'post_author': submission.author.name if submission.author else None,  # Handle deleted authors
    'post_text': submission.selftext.encode('utf-8').decode('utf-8'),
    'comments_bodies': [],
    'comments_lenght': [],
    'comments_author': [],
    'comments_upvote_score': [],
    'comments_created_utc': [],
    'comments_num': 0,
    'post_created_utc': datetime.utcfromtimestamp(submission.created_utc).isoformat(),
    'post_upvote_score': submission.score,
    'post_upvote_ratio': submission.upvote_ratio,
    'post_url': submission.url, 
    # 'post_language': language, 
}
        

# To fetch deeply nested comments:
submission.comments.replace_more(limit=None)

def print_comments(comment, level=0):
  """
  Recursively prints comments with indentation to show nesting.

  Args:
    comment: The comment object to start from.
    level: The current nesting level (for indentation).
  """
  print("  " * level + "- " + comment.body)
  for reply in comment.replies:
    print_comments(reply, level + 1)

# Now you can iterate through the comments:
for top_level_comment in submission.comments:
  print_comments(top_level_comment)

Fetching: GET https://oauth.reddit.com/comments/3g1jfi/ at 1730101160.2526844
Data: None
Params: {'limit': 2048, 'raw_json': 1, 'sort': 'confidence'}
Response: 200 (75294 bytes) (rst-38:rem-887.0:used-113 ratelimit) at 1730101162.8125622
Fetching: POST https://oauth.reddit.com/api/morechildren/ at 1730101162.864215
Data: [('api_type', 'json'),
 ('children',
  'ctu5sjx,ctud1hd,ctu38pi,ctu0mk8,ctu3jqd,ctuc5sn,ctuciai,ctu8qmv,ctu0791,ctu0lq1,ctu44um,cyikrzr,ctuc2pt,ctud8dw,ctudfza,ctu8dct,ctu85uv,ctuba3k,ctubqz6,ctu11wy,ctu1j06,ctu899f,ctu4s23'),
 ('link_id', 't3_3g1jfi'),
 ('sort', 'confidence')]
Params: {'raw_json': 1}
Response: 200 (5187 bytes) (rst-36:rem-886.0:used-114 ratelimit) at 1730101163.365923
Fetching: POST https://oauth.reddit.com/api/morechildren/ at 1730101163.3668692
Data: [('api_type', 'json'),
 ('children',
  'ctuc6vw,ctuceoh,ctuf0yj,ctucpmm,cvcrc69,ctul9rx,ctucymm,ctuyak2,cwu0a4e,ctudgos,ctums31,ctue7tu,ctur0ss,ctuecwj,ctuf4hv,ctutgbt,ctzz90y,ctui5nm,ctukmvi,cu0ia8m,cu

- [deleted]
  - Like the holy grail. 
    - ...And my GF :/
      - Nothing wrong with that.
        - The gf or her buttons?
          - Yes
            - WHY?!
              - As a programmer, I find Yes an acceptable answer here.
                -     from time import sleep as delay             


    while gf or buttons:                               


        print("Infinite Loop!")                          


        delay(1)                           


Edit: I hate formatting.
                  -     Traceback (most recent call last):
      File "<stdin>", line 1, in <module>
    NameError: name 'gf' is not defined
              - Yes because
                - Because because, because because, be-CAAAAUSE!
                  - because of the wonderful things she does.
          - He inadvertently pushes hers. 
          - Yes
        - [Nothing wrong with that!](https://youtu.be/wx7s_N79nco?t=2m10s)
          - I prefer just beating them
            - Nothing wrong with that.
 

In [33]:
import logging

stream_handler = logging.StreamHandler()
# stream_handler.setLevel(logging.DEBUG)
# file_handler = logging.handlers.RotatingFileHandler(
#     "praw_log.txt", maxBytes=1024 * 1024 * 16, backupCount=5
# )
# file_handler.setLevel(logging.DEBUG)
# # for logger_name in ("praw", "prawcore"):
# #     logger = logging.getLogger(logger_name)
# #     logger.setLevel(logging.DEBUG)
# #     logger.addHandler(stream_handler)
# #     logger.addHandler(file_handler)

In [ ]:
# Multi thread? 
https://github.com/praw-dev/praw/blob/master/docs/getting_started/multiple_instances.rst
